In [1]:
# Imports 
import numpy as np
import pandas as pd
import pyodbc as db
import matplotlib.pyplot as plt
import os
%matplotlib inline
import requests
from requests.auth import HTTPBasicAuth
from io import StringIO
from sodapy import Socrata
from datetime import date
import datetime
import seaborn as sns
import configparser

# Constants 
stringVAERSDATA     = "VAERSDATA"
stringVAERSSYMPTOMS = "VAERSSYMPTOMS"
stringVAERSVAX      = "VAERSVAX"
inputDirectory      = ".\RAW_DATA"
reworkedInputs      = ".\REWORKED_DATA"
outputDirectory     = ".\OUTPUT"
pklInputDirectory   = ".\INPUT"

In [2]:
# @TODO Column datatypes - need to finish  - getting the following warning when reading in files:
# DtypeWarning: Columns (11,13,15,16,27) have mixed types.Specify dtype option on import or set low_memory=False. interactivity=interactivity, compiler=compiler, result=result)
dtype_VAERSDATA = {"VAERS_ID": np.int64, "RECVDATE": np.object, "STATE": np.str, "AGE_YRS": np.float64, "CAGE_YR": np.float64, "CAGE_MO": np.float64, "SEX": np.str, "RPT_VDATE": np.object, "SYMPTOM_TEXT": np.str, "DIED": np.str, "DATEDIED": np.object, "L_THREAD": np.str, "ER_VISIT": np.str}

# Iterate over files 
for filename in os.listdir(inputDirectory):

    # Create the file name 
    fullFilename = os.path.join(inputDirectory, filename)

    # Ensure its a file
    if os.path.isfile(fullFilename):

        # Overall
        if(fullFilename.__contains__(stringVAERSDATA)):

            # Read in the file to this dataframe directly 
            dfVAERSDATA = pd.read_csv(fullFilename, encoding='cp1252', dtype=dtype_VAERSDATA)

            # Pull out all the commas from the string columns 
            for name, value in dfVAERSDATA.iteritems():
                if(dfVAERSDATA.dtypes[name] == "O"):
                    dfVAERSDATA[name] = dfVAERSDATA[name].str.replace(",", "!!!")

            # Drop the symptom text column to reduce the total size 
            dfVAERSDATA = dfVAERSDATA.drop("SYMPTOM_TEXT", axis="columns")

            # Save out to file now 
            dfVAERSDATA.to_csv(reworkedInputs + "\\" + filename, index=False)

        # Symptoms
        elif(fullFilename.__contains__(stringVAERSSYMPTOMS)):
                         
            # Read in the file to this dataframe directly 
            dfVAERSSYMPTOMS = pd.read_csv(fullFilename, encoding='cp1252')

            # Pull out all the commas from the string columns 
            for name, value in dfVAERSSYMPTOMS.iteritems():
                if(dfVAERSSYMPTOMS.dtypes[name] == "O"):
                    dfVAERSSYMPTOMS[name] = dfVAERSSYMPTOMS[name].str.replace(",", "!!!")

            # Save out to file now 
            dfVAERSSYMPTOMS.to_csv(reworkedInputs + "\\" + filename,  index=False)

        # Vax
        elif(fullFilename.__contains__(stringVAERSVAX)):
                
            # Read in the file to this dataframe directly 
            dfVAERSVAX = pd.read_csv(fullFilename, encoding='cp1252')

            # Pull out all the commas from the string columns 
            for name, value in dfVAERSVAX.iteritems():
                if(dfVAERSVAX.dtypes[name] == "O"):
                    dfVAERSVAX[name] = dfVAERSVAX[name].str.replace(",", "!!!")

            # Save out to file now 
            dfVAERSVAX.to_csv(reworkedInputs + "\\" + filename,  index=False)

C:\Users\IanD\AppData\Local\Temp/ipykernel_13632/1860838712.py:3: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype_VAERSDATA = {"VAERS_ID": np.int64, "RECVDATE": np.object, "STATE": np.str, "AGE_YRS": np.float64, "CAGE_YR": np.float64, "CAGE_MO": np.float64, "SEX": np.str, "RPT_VDATE": np.object, "SYMPTOM_TEXT": np.str, "DIED": np.str, "DATEDIED": np.object, "L_THREAD": np.str, "ER_VISIT": np.str}
C:\Users\IanD\AppData\Local\Temp/ipykernel_13632/1860838712.py:3: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; 

In [3]:
# Break apart the big files - PROBABLY DO NOT NEED THIS ANYMORE NOW THAT WE ARE REMOVING THE SYMPTOM_TEXT COLUMN
hugeFile = pd.read_csv(reworkedInputs + "\\NonDomesticVAERSDATA.csv")

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (7,11,22,26,29,30,31,32,33) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:

# Split the file 
hugeFileOne = hugeFile.iloc[ : int((hugeFile.shape[0] / 2)), : ]
hugeFileTwo = hugeFile.iloc[ int(((hugeFile.shape[0] / 2) + 1)) : , : ]
hugeFileOne.to_csv(reworkedInputs + "\\NonDomesticVAERSDATA_FIRST_HALF.csv",  index=False)
hugeFileTwo.to_csv(reworkedInputs + "\\NonDomesticVAERSDATA_SECOND_HALF.csv",  index=False)

In [22]:
# THIS IS DEVELOPMENT BELOW THIS CELL INCLUSIVE 

# Read in two files for 2021
dfVAERSDATA = pd.read_csv("C:\\Users\\IanD\\Documents\\GitHub\\VAERS\\REWORKED_DATA\\2021VAERSDATA.csv")
dfVAERSVAX = pd.read_csv("C:\\Users\\IanD\\Documents\\GitHub\\VAERS\\REWORKED_DATA\\2021VAERSVAX.csv")

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (7,12,15,23) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [23]:
# Merge the vax with the symptoms tables have duplicates or can 
# have duplicates hence using full outer join. Merging vax and symptoms first.
completeDataFrame = pd.merge(left = dfVAERSVAX, right = dfVAERSDATA, \
                      how = "outer", left_on = ["VAERS_ID"], \
                      right_on = ["VAERS_ID"])

In [24]:
# Get all the values of the type of vaccine to try and find the covid 
covidFrame = completeDataFrame[completeDataFrame["VAX_TYPE"] == "COVID19"]

# Get all the deaths per manufacturer
totalDeaths = (covidFrame[covidFrame["DIED"] == "Y"].drop_duplicates(subset=["VAERS_ID"]))

In [25]:
totalDeaths.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9810 entries, 502 to 792434
Data columns (total 42 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   VAERS_ID         9810 non-null   int64  
 1   VAX_TYPE         9810 non-null   object 
 2   VAX_MANU         9810 non-null   object 
 3   VAX_LOT          7001 non-null   object 
 4   VAX_DOSE_SERIES  9788 non-null   object 
 5   VAX_ROUTE        6725 non-null   object 
 6   VAX_SITE         5667 non-null   object 
 7   VAX_NAME         9810 non-null   object 
 8   RECVDATE         9810 non-null   object 
 9   STATE            7715 non-null   object 
 10  AGE_YRS          8767 non-null   float64
 11  CAGE_YR          8434 non-null   float64
 12  CAGE_MO          12 non-null     float64
 13  SEX              9810 non-null   object 
 14  RPT_DATE         19 non-null     object 
 15  SYMPTOM_TEXT     9806 non-null   object 
 16  DIED             9810 non-null   object 
 17  DATEDIED  

In [15]:
dfVAERSDATA.info()
dfVAERSDATA["DIED"].unique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 739949 entries, 0 to 739948
Data columns (total 35 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   VAERS_ID      739949 non-null  int64  
 1   RECVDATE      739949 non-null  object 
 2   STATE         644706 non-null  object 
 3   AGE_YRS       658823 non-null  float64
 4   CAGE_YR       587261 non-null  float64
 5   CAGE_MO       4109 non-null    float64
 6   SEX           739949 non-null  object 
 7   RPT_DATE      514 non-null     object 
 8   SYMPTOM_TEXT  739638 non-null  object 
 9   DIED          9917 non-null    object 
 10  DATEDIED      8706 non-null    object 
 11  L_THREAT      11256 non-null   object 
 12  ER_VISIT      86 non-null      object 
 13  HOSPITAL      47057 non-null   object 
 14  HOSPDAYS      31242 non-null   float64
 15  X_STAY        387 non-null     object 
 16  DISABLE       12188 non-null   object 
 17  RECOVD        668030 non-null  object 
 18  VAX_

array([nan, 'Y'], dtype=object)